In [5]:
from netCDF4 import Dataset
import numpy as np

In [22]:
filename='Slab2Distribute_Mar2018/him_slab2_dep_02.24.18.grd'
nc_fid=Dataset(filename,'r')

In [23]:
def extract_contour(dataset):
    
    from scipy.interpolate import NearestNDInterpolator
    from skimage.filters import gaussian
    from skimage import measure

    x = dataset.variables['x'][:]
    y = dataset.variables['y'][:]
    z = dataset.variables["z"][:]
       
    mask = z.mask
    X, Y = np.meshgrid(x, y)
    x = X[mask == False]
    y = Y[mask == False]
    z = z[mask == False]    
    
    interp = NearestNDInterpolator(list(zip(x, y)), z)
    
    values = np.where(mask, 0, 1) 
    for val in range(5):
        values = gaussian(values, 3, preserve_range=True) 
    
    # Contour should have length 1
    contour = measure.find_contours(values, 0.5)
    i = np.round(contour[0][:,0]).astype("int")
    j = np.round(contour[0][:,1]).astype("int")
         
    cx = X[i,j]
    cy = Y[i,j]
    cz = interp(cx, cy)            
                 
    return cx, cy, cz


def simplify_contour(x, y, z):
    from shapely.geometry import LinearRing 
    contour = LinearRing(zip(x, y, z))
    contour = contour.simplify(0.05)
    return contour.coords.xy


def get_interpolator(dataset):
    from scipy.interpolate import interp2d, griddata
    
    x = dataset.variables['x'][:]
    y = dataset.variables['y'][:]
    z = dataset.variables["z"][:]    
    
    X, Y = np.meshgrid(x, y)
    xi = X[z.mask == False]
    yi = Y[z.mask == False]
    zi = z[z.mask == False]
    
    z = griddata((xi, yi), zi, (x[None,:], y[:,None]), method='nearest')
    
    return interp2d(x, y, z)
    

In [24]:
xi, yi, zi = extract_contour(nc_fid)
x1, y1 = simplify_contour(xi, yi, zi)
f = get_interpolator(nc_fid)

## Mesh the extend of the slab


In [25]:

import gmsh
gmsh.initialize()
gmsh.model.add("test")

#gmsh.option.setNumber("Mesh.MeshSizeMax", 0.1)

In [26]:
pts_list = []
for idx in range(len(x1)-1):
        pt = gmsh.model.geo.add_point(x1[idx], y1[idx], 0)
        pts_list.append(pt)
        
if pts_list:
    pts_list = pts_list + [pts_list[0]]
    line_list = []
    for idx in range(len(pts_list)-1):
        pt1 = pts_list[idx]
        pt2 = pts_list[idx+1]
        line_list.append(gmsh.model.geo.add_line(pt1, pt2))

In [27]:
cl = gmsh.model.geo.add_curve_loop(line_list)
gmsh.model.geo.add_plane_surface([cl])

1

In [28]:
gmsh.model.geo.synchronize()

In [29]:
gmsh.model.mesh.generate(2)

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 3 (Line)
Info    : [ 10%] Meshing curve 4 (Line)
Info    : [ 10%] Meshing curve 5 (Line)
Info    : [ 20%] Meshing curve 6 (Line)
Info    : [ 20%] Meshing curve 7 (Line)
Info    : [ 20%] Meshing curve 8 (Line)
Info    : [ 20%] Meshing curve 9 (Line)
Info    : [ 30%] Meshing curve 10 (Line)
Info    : [ 30%] Meshing curve 11 (Line)
Info    : [ 30%] Meshing curve 12 (Line)
Info    : [ 30%] Meshing curve 13 (Line)
Info    : [ 40%] Meshing curve 14 (Line)
Info    : [ 40%] Meshing curve 15 (Line)
Info    : [ 40%] Meshing curve 16 (Line)
Info    : [ 40%] Meshing curve 17 (Line)
Info    : [ 40%] Meshing curve 18 (Line)
Info    : [ 50%] Meshing curve 19 (Line)
Info    : [ 50%] Meshing curve 20 (Line)
Info    : [ 50%] Meshing curve 21 (Line)
Info    : [ 50%] Meshing curve 22 (Line)
Info    : [ 60%] Meshing curve 23 (Line)
Info    : [ 60%] Meshing curve 24 (Line)
I

In [30]:
#gmsh.fltk.run()

## Advect Mesh in the z-direction

### Extract Nodes and elements

In [31]:
nodeTags = {}
nodeCoords = {}
elementTypes = {}
elementTags = {}
elementNodeTags = {}

In [32]:
entities = gmsh.model.get_entities()

In [33]:
# get the nodes and elements
for e in entities:
    nodeTags[e], nodeCoords[e], _ = gmsh.model.mesh.getNodes(e[0], e[1])
    elementTypes[e], elementTags[e], elementNodeTags[e] = gmsh.model.mesh.getElements(e[0], e[1])

In [34]:
gmsh.model.mesh.clear()

### Advect and add elements

In [35]:
import numpy as np
import random
import math

for e in entities: 
    for i in range(2, len(nodeCoords[e]), 3):
        ii = nodeCoords[e][i-2]
        jj = nodeCoords[e][i-1]
        nodeCoords[e][i] = 0.05 * f(ii, jj)
    gmsh.model.mesh.addNodes(e[0], e[1], nodeTags[e], nodeCoords[e])
    gmsh.model.mesh.addElements(e[0], e[1], elementTypes[e], elementTags[e],
                               elementNodeTags[e])

In [36]:
gmsh.write("test.vtk")

Info    : Writing 'test.vtk'...
Info    : Done writing 'test.vtk'


In [37]:
gmsh.write("test.msh")

Info    : Writing 'test.msh'...
Info    : Done writing 'test.msh'


In [39]:
gmsh.open("test.msh")

Info    : Reading 'test.msh'...
Info    : 87 entities
Info    : 89 nodes
Info    : 219 elements
Info    : Done reading 'test.msh'


In [38]:
#gmsh.fltk.run()